# Assignment

## 우리의 목표 : 나이브베이즈 문제 해결하기
1.1) 입력문서가 {fast, furious, fun} 만을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 문서로 분류되는가?

##### 노트북 파일을 따라오면서, 빈칸과 질문에 대한 물음을 모두 채우시면 과제 완료 입니다 

### 주어진 dataset 을 확인해 보기위해 엑셀 파일을 읽습니다.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
documents = pd.read_excel('test_file.xlsx') 

In [ ]:
documents 

,label,message
0,comedy,"Fun couple, love love"
1,action,"fast Furious, shoot!!"
2,comedy,"couple^^, fly, fast, fun, fun"
3,action,"furious, shoot shoot fun"
4,action,"fly~~ fast shoot, Love"


####  Q1. **문제점**(전처리가 필요한 부분)을 2가지 이상 적어주세요.
- 1. 특수문자, 대소문자 섞임
- 2. 단어와 단어의 구분이 모호함

### step 1. 단어를 모두 소문자로 바꾸어 줍니다. 

#### Q2. 코드속 빈칸을 채워주세요

In [ ]:
lower_documents = []
lower_documents = [d.lower() for d in documents['message']]
print(lower_documents)

['fun couple, love love', 'fast furious, shoot!!', 'couple^^, fly, fast, fun, fun', 'furious, shoot shoot fun', 'fly~~ fast shoot, love']


### step2. 특수문자를 없애줍니다.
#### string.punctuation 에는 특수문자들이 저장되어있는것을 확인할수 있습니다.

In [ ]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### Q3. string.punctuation 을 이용해서 특수문자를 제거해주세요.

In [ ]:
import re
import string

del_punctuation_documents = []

for i in lower_documents:
  del_punctuation_documents.append(i.translate(str.maketrans("","",string.punctuation)))
    
del_punctuation_documents

['fun couple love love',
 'fast furious shoot',
 'couple fly fast fun fun',
 'furious shoot shoot fun',
 'fly fast shoot love']

### step 3. 단어를 하나씩 띄어쓰기 단위로 쪼개어 줍니다.
#### Q4. 빈칸을 채워주세요.

In [ ]:

preprocessed_documents = [[w for w in d.split(' ')] for d in del_punctuation_documents]

preprocessed_documents

[['fun', 'couple', 'love', 'love'],
 ['fast', 'furious', 'shoot'],
 ['couple', 'fly', 'fast', 'fun', 'fun'],
 ['furious', 'shoot', 'shoot', 'fun'],
 ['fly', 'fast', 'shoot', 'love']]

### step 4. 각각의 단어가 문서에서 몇번 나왔는지 세봅시다.

In [ ]:
frequency_list = []
from collections import Counter

frequency_list = [Counter(d) for d in preprocessed_documents]
frequency_list

[Counter({'couple': 1, 'fun': 1, 'love': 2}),
 Counter({'fast': 1, 'furious': 1, 'shoot': 1}),
 Counter({'couple': 1, 'fast': 1, 'fly': 1, 'fun': 2}),
 Counter({'fun': 1, 'furious': 1, 'shoot': 2}),
 Counter({'fast': 1, 'fly': 1, 'love': 1, 'shoot': 1})]

### step 5. 이제 우리가 하고싶은것은, 문자를 숫자로 변환하는 작업입니다!(그림 참고)
- 이를 위해 CountVectorizer를 사용합니다.
- CountVectorizer는 문서 집합에서 단어 토큰을 생성하고, 각 단어 수를 세어 BOW (Bag of Words) 인코딩한 벡터를 만들어줍니다.

- 문서에 해당단어가 몇번 포함되었는지를 나타낼 때 사용하는 방법입니다.

- 참고 : https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/

![CountVectorized](https://user-images.githubusercontent.com/68625698/106378540-15d8ed80-63e9-11eb-8604-5c960c274867.PNG)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()
count_vector.fit(documents['message'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [ ]:
doc_array = count_vector.transform(documents['message']).toarray()
doc_array

array([[1, 0, 0, 1, 0, 2, 0],
       [0, 1, 0, 0, 1, 0, 1],
       [1, 1, 1, 2, 0, 0, 0],
       [0, 0, 0, 1, 1, 0, 2],
       [0, 1, 1, 0, 0, 1, 1]])

In [ ]:
count_vector.get_feature_names()

['couple', 'fast', 'fly', 'fun', 'furious', 'love', 'shoot']

In [ ]:
count_vector.vocabulary_

{'couple': 0,
 'fast': 1,
 'fly': 2,
 'fun': 3,
 'furious': 4,
 'love': 5,
 'shoot': 6}

In [ ]:
frequency_matrix = pd.DataFrame(doc_array, columns = count_vector.get_feature_names())
frequency_matrix

,couple,fast,fly,fun,furious,love,shoot
0,1,0,0,1,0,2,0
1,0,1,0,0,1,0,1
2,1,1,1,2,0,0,0
3,0,0,0,1,1,0,2
4,0,1,1,0,0,1,1


In [ ]:
frequency_matrix['count'] = frequency_matrix.sum(axis=1)

### step 6. 범주형 변수를 dummy변수로 변환해주는 작업(One-Hot Encoding!)을 해주어야합니다.
#### Q5. label 을 comedy =1, action =0 으로 변환해주세요

In [ ]:
documents["label"]=pd.get_dummies(documents["label"])

In [ ]:
doc = pd.concat([documents['label'],frequency_matrix],axis=1)

In [ ]:
doc

,label,couple,fast,fly,fun,furious,love,shoot,count
0,1,1,0,0,1,0,2,0,4
1,0,0,1,0,0,1,0,1,3
2,1,1,1,1,2,0,0,0,5
3,0,0,0,0,1,1,0,2,4
4,0,0,1,1,0,0,1,1,4


###  step 7. 나이브 베이즈 계산을 해봅시다!

#### Q6. 입력문서가 {fast, furious, fun} 을 주요 단어로 가질때, 이 문서는 얼마의 확률로 어떤 문서로 분류가 될까요? ( 계산과정을 채워주세요) 

In [ ]:
doc =  np.array(doc)        # dataframe을 np-array로 변환해줍니다.
                         # return값은 np-matrix가 아니라 np-array입니다.
doc

array([[1, 1, 0, 0, 1, 0, 2, 0, 4],
       [0, 0, 1, 0, 0, 1, 0, 1, 3],
       [1, 1, 1, 1, 2, 0, 0, 0, 5],
       [0, 0, 0, 0, 1, 1, 0, 2, 4],
       [0, 0, 1, 1, 0, 0, 1, 1, 4]])

In [ ]:
type(doc)

numpy.ndarray

####  P(Y=comedy), P(Y=action) 계산하기



*   분모 : 전체 Y의 경우 : 행의 길이 (5개)
*   분자 <br>
: comedy인 경우 : label 행의 값이 1인 경우 (2개) <br>
: action인 경우 : label 행의 값이 0인 경우 (3개)



In [ ]:
# P(Y=comedy)=2/5
p_comedy = sum(doc[:,0]==1) / len(doc)

# P(Y=action)=3/5
p_action = sum(doc[:,0]==0) / len(doc)

In [ ]:
print('p_comedy : ',p_comedy)
print('p_action : ', p_action)

p_comedy :  0.4
p_action :  0.6


#### P(fast=1|comedy=1), P(furious=1|comedy=1), P(fun=1|comedy=1) 계산하기
* 참고 : 문서에 두번 등장한 단어 주의


*   분모<br> : y = comedy인 경우로 한정 : label행의 값이 1인 것에 마지만 count로 sum하면 총 단어의 갯수
*   분자<br> : comedy와 특정 단어 : label행의 값이 comedy를 만족하는 것을 indexing하고 특정 단어가 포함된 열을 indexing해서 그 합이 갯수

**단어가 2번 특장하는 경우: 특정 칸 ==1이 아닌 특정 칸!=1로 표현 **


In [ ]:
# P(fast=1|comedy=1)
p_comedy_fast = sum(doc[(doc[:,2]!=0) &(doc[:,0]!=0)][:,2]) / sum(doc[doc[:,0]!=0][:,8])

# P(furious=1|comedy=1)
p_comedy_furious = sum(doc[(doc[:,5]!=0) & (doc[:,0]!=0)][:,5]) / sum(doc[doc[:,0]!=0][:,8]) 

# P(fun=1|comedy=1)
p_comedy_fun = sum( doc[(doc[:,4]!=0) & (doc[:,0]!=0)][:,4]) / sum(doc[doc[:,0]!=0][:,8])

In [ ]:
print('p_comedy_fast : ' , p_comedy_fast)
print('p_comedy_furious : ' , p_comedy_furious)
print('p_comedy_fun : ' , p_comedy_fun)   

p_comedy_fast :  0.1111111111111111
p_comedy_furious :  0.0
p_comedy_fun :  0.3333333333333333


#### P(fast=1|action=1), P(furious=1|action=1), P(fun=1|action=1) 계산하기
* 참고 : 문서에 두번 등장한 단어 주의


*   분모 : y = action인 경우로 한정 : label행의 값이 0인 것에 마지만 count로 sum하면 총 단어의 갯수
*   분자 : action와 특정 단어 : label행의 값이 action를 만족하는 것을 indexing하고 특정 단어가 포함된 열을 indexing해서 그 합이 갯수

**단어가 2번 특장하는 경우: 특정 칸 ==1이 아닌 특정 칸!=1로 표현 **


In [ ]:
# P(fast=1|action=1)
p_action_fast = sum(doc[(doc[:,2]!=0) &(doc[:,0]!=1)][:,2]) / sum(doc[doc[:,0]!=1][:,8])

# P(furious=1|action=1)
p_action_furious = sum(doc[(doc[:,5]!=0) & (doc[:,0]!=1)][:,5]) / sum(doc[doc[:,0]!=1][:,8]) 

# P(fun=1|action=1)
p_action_fun = sum( doc[(doc[:,4]!=0) & (doc[:,0]!=1)][:,4]) / sum(doc[doc[:,0]!=1][:,8])

In [ ]:
print('p_action_fast : ' , p_action_fast)
print('p_action_furious : ' , p_action_furious)
print('p_action_fun : ' , p_action_fun)  

p_action_fast :  0.18181818181818182
p_action_furious :  0.18181818181818182
p_action_fun :  0.09090909090909091


#### P(Y = comedy| X = fast, furious, fun) , P(Y = action=1| X = fast, furious, fun) 값 계산하기

베이즈 정리 / 조건부 독립 공식 -> 분모를 직접 구하지 않고 오직 곱셈만을 이용해 다음의 값을 구함

In [ ]:
#P(Y = comedy| X = fast, furious, fun)
proba_comedy = p_comedy*p_comedy_fast*p_comedy_furious*p_comedy_fun

#P(Y = action=1| X = fast, furious, fun)
proba_action = p_action*p_action_fast*p_action_furious*p_action_fun

In [ ]:
print('proba_comedy', proba_comedy)
print('proba_action', proba_action)

proba_comedy 0.0
proba_action 0.001803155522163787


### step 8. 다음 값을 비교해봅시다.
1. P(Y = comedy| X = fast, furious, fun)
2. P(Y = action=1| X = fast, furious, fun)


Q7. 어떤 문제점을 발견할수 있나요? 문제점을 해결하기 위한 방법으로는 어떤게 있을까요?
- A.<br>
문제점 : p_comedy_furious = 0으로 곱셈을 취하여 proba_comedy을 구하면 proba_comedy값 또한 0이 됨<br>
해결법 : `라플라스 스무딩` : 가능성(likelihood)가 0이 되는 것을 방지하고자 최소한의 확률을 정함<br>-> 분자 : 1 더하기 <br> ->분모 : 입력 변수들의 갯수 더하기 (관찰한 단어 갯수)

In [ ]:
v=(len(doc[0])-2) #입력한 변수들의 갯수 

# P(fast=1|comedy=1)
p_comedy_fast_2 = (sum(doc[(doc[:,2]!=0) &(doc[:,0]!=0)][:,2])+1) / (sum(doc[doc[:,0]!=0][:,8])+v)

# P(furious=1|comedy=1)
p_comedy_furious_2 = (sum(doc[(doc[:,5]!=0) & (doc[:,0]!=0)][:,5])+1) / (sum(doc[doc[:,0]!=0][:,8])+v)

# P(fun=1|comedy=1)
p_comedy_fun_2 = (sum( doc[(doc[:,4]!=0) & (doc[:,0]!=0)][:,4])+1) / (sum(doc[doc[:,0]!=0][:,8])+v)


# P(fast=1|action=1)
p_action_fast_2 = (sum(doc[(doc[:,2]!=0) &(doc[:,0]!=1)][:,2])+1) / (sum(doc[doc[:,0]!=1][:,8])+v)

# P(furious=1|action=1)
p_action_furious_2 = (sum(doc[(doc[:,5]!=0) & (doc[:,0]!=1)][:,5])+1) / (sum(doc[doc[:,0]!=1][:,8])+v)

# P(fun=1|action=1)
p_action_fun_2 = (sum( doc[(doc[:,4]!=0) & (doc[:,0]!=1)][:,4])+1) / (sum(doc[doc[:,0]!=1][:,8])+v)

In [ ]:
#P(Y = comedy| X = fast, furious, fun)
proba_comedy_2 = p_comedy*p_comedy_fast_2*p_comedy_furious_2*p_comedy_fun_2
#P(Y = action=1| X = fast, furious, fun)
proba_action_2 = p_action*p_action_fast_2*p_action_furious_2*p_action_fun_2

In [ ]:
print('proba_comedy', proba_comedy_2)
print('proba_action', proba_action_2)

proba_comedy 0.00078125
proba_action 0.0018518518518518513


proba_action > proba_comedy<br>fast, furious, fun이 주요단어로 포함된 경우 action으로 분류될 가능성이 높음